In [1]:
import gradio as gr
from typing import Optional, List, Any
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextIteratorStreamer, AutoProcessor
# from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.outputs.generation import GenerationChunk
from torch import device
import time
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.history_aware_retriever import create_history_aware_retriever

from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain

from transformers import pipeline


from threading import Thread
import warnings
warnings.filterwarnings("ignore")

/root/miniconda3/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/root/miniconda3/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issu

# 该部分代码实现
## 1.历史记录聊天
## 2.实体提取
## 3.意图识别
## 4.主体模型gemma-2-9B-it

In [31]:
# 构建模型
class QianWenLLM(LLM):
    # 基于本地的QianWen7B-Chat模型自定义LLM类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None
    processor: AutoProcessor = None
    
    def __init__(self, model_dir: str):
        # 从本地加载模型
        super().__init__()
        print('正从本地加载模型。。。。。')

        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            trust_remote_code=True,
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            device_map='auto',
            trust_remote_code=True,
            # torch_dtype=torch.bfloat16,
            # temperature=0.1
            )
        self.model = self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(
            model_dir,
            trust_remote_code=True
        )
        # 可指定不同的生成长度、top_p等相关超参
        self.processor = AutoProcessor.from_pretrained(model_dir)
        print('模型加载完成！')

    def _call(self, messages,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # response, history = self.model.chat(self.tokenizer, prompt, history=[])
        if type(messages) == str:
            system_info =  messages.split('System:')[-1].split('Human:')[0].strip()
            human_info =  messages.split('Human:')[-1].strip()
            re_prompt = [
                    {'role':'system', 'content': system_info},
                    {'role':'user', 'content': human_info}
                    ]
            messages = re_prompt
        # print('打印messages内容：', messages)
        # print('打印messages类型：', type(messages))
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors='pt').to('cpu')
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        # print('response, messages', response, '\n', messages)
        return response
        
    @property
    def _llm_type(self) -> str:
        return "QwenLM"

In [32]:
# 构建改写问题prompt

# 请根据聊天历史和最后用户的问题，改写用户最终剔除的问题。
# 你只需要改写用户最终的问题，请不要回答问题。
# 没有聊天历史则将用户问题直接返回，有聊天历史则改写。
    
def contextualize_question_prompt():
    system_prompt = """\
    根据聊天历史和输入文本，改写输入文本。请根据工作流程和工作要求改写文本。

    工作流程：
    1-判断聊天历史是否为空。内容为空直接返回输入文本，否则继续以下工作流程。
    2-聊天历史存在内容，请根据聊天历史和输入文本内容相关程度，改写输入文本。如果相关程度低则直接输出文本。
    3-请思考改写后的输入文本内容，是否突出文本意图，否则重新根据输入文本改写。请注意如果是打招呼等日常用语也直接返回输入文本。
    4-请不要回答输入的文本，输出改写的文本内容。

    工作要求：
    1-你只需要改写文本。
    2-聊天历史为空则将输入文本直接返回，有聊天历史则改写。    
    """
    contextualize_question_prompt = ChatPromptTemplate(
        [
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}')
        ]
    )
    return contextualize_question_prompt

def answer_prompt():
    # 构建正常问答prompt
    system_prompt = """\
    使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。
    {context}
    """
    # 问题: {question}
    # 有用的回答:
    
    qa_prompt = ChatPromptTemplate(
    [
    ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}')
    ]
    )
    return qa_prompt

store = {}
# 构造存储函数
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [33]:
def init_model():
    # 初始化模型
    # model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    model_cache_path = r'autodl-tmp/LLM-Research/gemma-2-9b-it'
    llm = QianWenLLM(model_dir=model_cache_path)
    return llm

In [34]:
def init_db():
    persist_directory = r'vectordb/chroma'
    embeddings_model_cache_path = r'autodl-tmp/embedding_model/Ceceliachenen/paraphrase-multilingual-MiniLM-L12-v2'
    # 加载词向量模型
    embeddings = HuggingFaceEmbeddings(
        model_name=embeddings_model_cache_path)
    # 加载缓存知识库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
    )
    return vectordb

In [35]:
def intention_recognition(question: str, llm):
    template = f"""
        你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
        
        任务：
        1-判断用户问题是否存在实体。
        2-抽取用户问题所有实体。
        3-根据实体与给出的意图标签进行判定该用户问题的意图。
        
        意图标签：政策知识，日常知识，招中标知识
        
        工作流程：
        1.-先判断是否存在实体，不存在实体则直接根据不存在实体输出格式输出，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
        2-实体提取：请从用户问题中提取出所有实体。
        3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类用户问题。
        
        不存在实体输出格式：
            实体提取:[]
            意图分类:日常知识
            
        存在实体输出格式：
            实体提取：[实体1, 实体2, ...]
            意图分类：意图标签
            
        有用的回答：
       """
          # 4. 意图分类：根据第3点的意图识别结果分类，意图标签：[政策知识，日常知识，招中标知识]。       2.意图识别：[意图1, 意图2, ...]。
    # ，例如日常、政策、设备、建筑、维修等
    messages = [
            {'role':'system', 'content': template},
            {'role':'user', 'content':question}]

    response = llm._call(messages)
    # print('实体+意图模型结果：', response)
    target = response.split('意图分类')[-1]
    # print('模型实体+意图结果：', response)
    print('意图分类:', target)
    return target
        

In [36]:
def qa_chain(rag_chain, get_session_history):
    """
    构建问答链
    :param persist_directory: 知识库本地保存路径，这里初始化政策信息知识库
    :return: 返回调用LLM回答
    """
    # 包装
    conversational_rag_chain = RunnableWithMessageHistory(
        runnable=rag_chain,
        get_session_history=get_session_history,
        input_messages_key='input',
        history_messages_key='chat_history',
        output_messages_key='answer',
    )

    return conversational_rag_chain


In [37]:
def contextualize_question_chain(llm):
    # 根据历史检索器原理，我们这里只要llm和改写prompt和历史记录就行
    question_prompt = contextualize_question_prompt()
    contextualize_question_chain = RunnableWithMessageHistory(
        question_prompt | llm,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history"
    )
    return contextualize_question_chain

In [38]:
# 链接前面的函数
class Model_center():
    """
      存储问答 Chain 的对象
    """

    def __init__(self, ):
        print('初始化模型+知识库。。。。')
        self.model = init_model()
        self.db = init_db()
         # 因为历史记录问题进行问题改写再传到模型
        self.question_prompt = contextualize_question_prompt()
        # 历史对话巡回器
        self.history_aware_retriever = create_history_aware_retriever(self.model, 
                                                                self.db.as_retriever(), 
                                                                 self.question_prompt)
        self.qa_prompt = answer_prompt()
        self.qa_chain = create_stuff_documents_chain(self.model, self.qa_prompt)
        self.rag_chain = create_retrieval_chain(self.history_aware_retriever, self.qa_chain)
        

    def qa_chain_self_answer(self, question: str, chat_history: list = []):
        # print('调用问答链')
        # print('打印用户问题', question)
        if question == None or len(question) < 1:
            print('问答为空。。。。')
            return '', chat_history
        print('实体提取+意图识别。。。。')

        # for question in sentence:
        # try
        # print('用户输入问题：', question)
        target = intention_recognition(question, self.model)
        # 只提取中文
        def isChinese(target):
            for ch in target: 
                if not '\u4e00' <= ch <= '\u9fff':
                    return False
            return True 
        if not isChinese(target):
            target=re.sub('[^\u4e00-\u9fa5]+','',target)
            print('意图识别存在非中文字符，调整后：', target)
        print('意图识别成功。。。。')
        # except:
            # print('实体提取+意图识别步骤失败。。。。')


        if target == '招中标知识' or target == '政策知识':
            print('调用检索问答链。。。。')
            # print('知识库:', target)
            # 带langchain实力对话共能
            response = qa_chain(
                self.rag_chain,
                get_session_history).invoke(
                    {'input': question},
                    config={'configurable':{'session_id':'test123'}}
                    )
            call = response['context']
            answer = response['answer']
            new_question = response['input']
            print('改写前的问题：', question, '改写后的问题：', new_question)
            print(f'意图分类：{target}，测试问题：{question}，返回结果：{answer}')
            # print('调用检索问答链输出：', response)
            # call = messages[0]['content'].split('不要试图编造答案。').strip()
            # print('检索链call：', call)

        else:
            
            print('调用简单模型。。。。')
            
            # 改写问题
            
            # 将改写的新问题输入进行回答，改写的记录也会加入聊天历史中
            new_question = contextualize_question_chain(self.model).invoke( 
                {'input': question},
                config={'configurable':{'session_id':'test123'}}
                    )
            # print('用户问题：', question)
            # print('根据历史聊天记录改写用户问题：', new_question)

            template = """\
                        请回答用户的问题，如果你不知道答案，就说你不知道，不要试图编造答案。
                    """
            messages = [
                {'role':'system', 'content': template},
                {'role':'user', 'content':new_question}
                ]
            response = self.model._call(messages)
            store['test123'].add_user_message(question)
            store['test123'].add_ai_message(response)
            call = ''
            print(f'意图分类：{target}，测试问题：{question}，返回结果：{response}')
            print('改写前的问题：', question, '改写后的问题：', new_question)
        return target, new_question, response, call
        
        # 检索问答链+历史聊天组件
        # response = self.qa_chain.invoke({'query': question, 'chat_history': chat_history})['answer']

        # chat_history.append([question, response])
        # print(chat_history)
        # return '', chat_history
                
        # except Exception as e:
        #     print('问答链报错', e)
        #     return e, chat_history

    def clear_history(self):
        self.qa_chain.clear_history()

In [ ]:
import pandas as pd
import csv
import os
import re
model_center = Model_center()
# 加载数据文本，提取问题和真实数据源
test_res = []
qa_data = pd.read_excel('问答对.xlsx').iloc[:, 2:]
length = len(qa_data.iloc[0, :])
history = []
# target_list = []
# new_question_list = []
# response_list = []
# call = []
file = 'qwen2.5模型测试结果.csv'
data_res = pd.DataFrame([], columns=['Question','Answer', 'reference', 'Metadata', 'Text', '意图分类','问题改写', '回答', 'RAG召回'])
if os.path.exists(file):
    data_tested = pd.read_csv(file, encoding='utf-8')['Question'].tolist()
else:
    data_tested = []
for q in range(len(qa_data)):
    question = qa_data.iloc[q, 0]
    
    if question not in data_tested:
        print('打印问题:', question)
        target, new_question, response, call = model_center.qa_chain_self_answer(question, history)
        data_ = qa_data.iloc[q].values.tolist() + [target] + [new_question] + [response] + [call]
        with open(file, 'a', newline='', encoding='utf-8') as f:
            fw = csv.writer(f)
            if not os.path.getsize(file):
                header = ['Question', 'Answer', 'reference', 'Metadata', 'Text', '意图分类', '问题改写', '回答', 'RAG召回']
                fw.writerow(header)
            fw.writerow(data_)
    #         # 写入标题行
    # writer.writerow(headers)
    # data_.to_excel('qwen2.5模型测试结果.xlsx', index=False)



初始化模型+知识库。。。。
正从本地加载模型。。。。。


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
sentence = [
            '你好',
            '你是谁？',
            '上海联合产权交易所有限公司物权交易操作指引(沪联产交〔2020〕26号)',
            '周杰伦是谁？',
            '产权交易争议调解',
            '2023年7月，中国航天科技集团公司五院研制的长征十二号运载火箭在酒泉卫星发射中心成功发射，将一颗北斗导航卫星送入预定轨道。',
            '周杰伦的歌曲《稻香》是谁写的？',
            '项目名称为ABC，中标金额为500万元，供应商为XYZ公司。']

In [ ]:
template = f"""
        你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
        
        任务：
        1-判断输入文本是否存在实体。
        2-抽取输入文本所有实体。
        3-根据实体与给出的意图标签进行判定该输入文本的意图。
        
        意图标签：政策知识，日常知识，招中标知识
        
        工作流程：
        1.-先判断是否存在实体，不存在实体则直接根据不存在实体输出格式输出，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
        2-实体提取：请从输入的文本中提取出所有实体。
        3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类输入文本。
        
        输入文本：{question}
        不存在实体输出格式：
            实体提取:[]
            意图分类:日常知识
        存在实体输出格式：
            实体提取：[实体1, 实体2, ...]
            意图分类：意图标签

        有用的回答：
       """

In [10]:
question_prompt = f"""\
    根据聊天历史和用户问题，改写用户问题。请根据工作流程和工作要求改写问题。

    工作流程：
    1-判断聊天历史是否为空。内容为空直接返回用户的问题，否则继续以下工作流程。
    2-聊天历史存在内容，请根据聊天历史和用户的问题，改写用户提出的问题。
    3-请思考改写后的问题内容，是否突出用户问题意图，否则重新根据用户问题改写。请注意如果是打招呼等日常用语也直接返回用户问题。
    4-请不要回答用户的问题，输出改写的问题。

    工作要求：
    1-你只需要改写用户的问题。
    2-聊天历史为空则将用户问题直接返回，有聊天历史则改写。                         

    聊天历史：{chat_history_}
    """

NameError: name 'chat_history_' is not defined

In [11]:
def history():
    MessagesPlaceholder('chat_history')

In [29]:
file = 'qwen2.5模型测试结果.csv'
data_tested = pd.read_csv(file, encoding='utf-8')['Question'].tolist()
data_tested

['如果投标文件解密失败，招标人是否有义务提供补救方案？',
 '如果投标文件解密失败，招标人是否有义务提供补救方案？',
 '如果投标文件解密失败，招标人是否有义务提供补救方案？']

In [17]:
# 只提取中文
target = "'_'.join(['招中标知识', '公示和公布要求'])"
def isChinese(target):
    for ch in target: 
        if not '\u4e00' <= ch <= '\u9fff':
            return False
    return True 
if not isChinese(target):
    target=re.sub('[^\u4e00-\u9fa5]+','',target)
    print('意图识别存在非中文字符，调整后：', target)


意图识别存在非中文字符，调整后： 招中标知识公示和公布要求


In [ ]:
['Question']